**Graphsage**

Graphsage is a inductive representation learning algorithm for large graphs.

Unlike transductive algorithms, which aim to learn the embeddings of each nodes, Graphsage aims to learn an aggregator functions instead.

As we know, **unsupervised algorithms** like deepwalk, node2vec aim to generate different walk sequences based on structual information of graph and then use skip-gram to learn the embedding of each node in the graph. These embeddings are not related to specific downstream task, so they could be general to many tasks. However, the ability to express the information containing in the graph is also limited. In this case, some authros proposed **GCN**, which is the first transductive algorithm using NN model. Each epoch, the model will update embeddings for each node. For large graphs, this usually could not be done.

Graphsage is designed to fix these flaws in the field of graph representation learning. Instead of directly learning node embeddings for each node, the authors decide to learn **aggregators** for nodes to get information from their neighbors and get the final embeddings.

Since we know, if we want to get as much information in the graph as we can, we should consider the structual information to generate embeddings for nodes. How to get **structual information**? Usually, we consider **k-hop neighbors** of nodes as their k-level structual information. Graphsage also use this idea. For each node, Graphsage tries to aggregate the features of its neighbors to get structual information. And then they combine neighbors' information and its own information to generate new information. This new information is the embedding for the node, containing structual information(neighbors' features) and own features.

First, they **sample a fixed number** of unique neighbors for each node(to speed up, and experiment shows it won't influence the performance). 

Second, they use aggregator to aggregate the features of these neighbors for each node. The authors proposed three types of aggregators, **mean, max-pooling, and LSTM**. The core idea of designing aggregator is to make sure it's **symmetric** while training because it has to operate over an unordered set of vectors.

Third, we get fused neighbors' features for each node. We use this fused feature to **combine** with the feature of center node and **project** this combined feature to a **lower** space to reduce the dimension of each node.

Fourth, iterate step 1 to 3 many times and use loss function to update weights of aggregators and projectors, we could get final model. We could train this model as unsupervised or supervised model. Below is the example of supervised model.

In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric 

Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 7.9 MB 5.3 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 3.5 MB 5.4 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 2.5 MB 5.4 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
     |████████████████████████████████| 750 kB 4.6 MB/s 
     |████████████████████████████████| 407 kB 5.5 MB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616603 sha256=7818cc903c48b72d2b005a2d909b6dd779753d4d202e7e9f591f096a1465ec24
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [ ]:
from torch_geometric.datasets import Planetoid
import os
import re
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from collections import defaultdict
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
dataset = Planetoid(root='/tmp/Cora', name='Cora')
nodenum = dataset.data.num_nodes
edges = dataset.data.edge_index.T
nodes = defaultdict(lambda: defaultdict(int))
for edge in edges:
  nodes[edge[0].item()][edge[1].item()] += 1
  nodes[edge[1].item()][edge[0].item()] += 1

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = dataset[0].to(device)
#features = data.x
features = {}
for i in range(data.x.shape[0]):
  features[i] = data.x[i].unsqueeze(0)

In [ ]:
class alias():
  def __init__(self, probs):
    self.n = len(probs)
    self.scaledprobs = {}
    self.table = {}
    self.aliastable = {}
    self.small = []
    self.big = []
    self.keys = list(probs.keys())

    for item in probs:
      prob = probs[item]
      self.scaledprobs[item] = prob * self.n
      if self.scaledprobs[item] > 1:
        self.big.append(item)
      elif self.scaledprobs[item] < 1:
        self.small.append(item)
      else:
        self.table[item] = 1
    
    while self.small and self.big:
      smallitem = self.small.pop()
      bigitem = self.big.pop()
      newprob = self.scaledprobs[bigitem] - (1 - self.scaledprobs[smallitem])
      self.table[smallitem] = self.scaledprobs[smallitem]
      self.aliastable[smallitem] = bigitem
      self.scaledprobs[bigitem] = newprob
      if self.scaledprobs[bigitem] > 1:
        self.big.append(bigitem)
      elif self.scaledprobs[bigitem] < 1:
        self.small.append(bigitem)
      else:
        self.table[bigitem] = 1
    
    while self.small:
      smallitem = self.small.pop()
      self.table[smallitem] = 1
    
    while self.big:
      bigitem = self.big.pop()
      self.table[bigitem] = 1

  def sampling_one(self):
    sample = random.choice(self.keys)
    if self.table[sample] >= random.uniform(0, 1):
      return sample
    else:
      return self.aliastable[sample]
  
  def sampling_n(self, n):
    samples = []
    for i in range(n):
      samples.append(self.sampling_one())
    return samples

In [ ]:
neighbors = {}
for node in nodes:
  neighbors[node] = alias(nodes[node])

In [ ]:
batchsz = 64

In [ ]:
xs = torch.IntTensor(nodenum, 1)
for i in range(nodenum):
  xs[i] = i

In [ ]:
trainset = torch.utils.data.TensorDataset(xs[data.train_mask].to(device), data.y[data.train_mask])
train_loader = DataLoader(trainset, batch_size=batchsz, shuffle=True)

In [ ]:
class Graphsage(nn.Module):
  def __init__(self, neighbors, features, k, n, dims):
    super(Graphsage, self).__init__()
    self.neighbors = neighbors
    self.features = features
    self.k = k
    self.n = n
    self.aggregator = nn.ModuleList([nn.Linear(dims[i], dims[i], bias=True) for i in range(self.k)])
    self.linears = nn.ModuleList([nn.Linear(2 * dims[i - 1], dims[i], bias=False) for i in range(1, self.k + 1)])
    self.mlp = nn.Linear(dims[-2], dims[-1])
    self.bns = nn.ModuleList([nn.BatchNorm1d(dims[i]) for i in range(1, self.k + 1)])
    self.relu = nn.ReLU()
    self.softmax = nn.LogSoftmax()

  def forward(self, batch):
    batchs = defaultdict(set)
    batchs[self.k] = set()
    batchneighbors = defaultdict(lambda: defaultdict(list))
    batchlist = []
    
    for node in batch:
      batchs[self.k].add(node.item())
      batchlist.append(node.item())

    for i in range(self.k, 0, -1):
      batchs[i - 1] |= batchs[i]
      for node in batchs[i]:
        kneighbors = set(self.neighbors[node].sampling_n(self.n))
        batchs[i - 1] |= kneighbors
        batchneighbors[i - 1][node] = list(kneighbors)

    fs = deepcopy(self.features)

    for i in range(1, self.k + 1):
      aggregate = {}
      for node in batchs[i]:
        for neigh in batchneighbors[i - 1][node]:
          if node not in aggregate:
            aggregate[node] = fs[neigh]
          else:
            aggregate[node] = torch.cat((aggregate[node], fs[neigh]), dim=0)
        aggregate[node] = torch.mean(aggregate[node], dim=0, keepdim=True)
        aggregate[node] = self.aggregator[i - 1](aggregate[node])
        aggregate[node] = torch.cat((aggregate[node], fs[node]), dim=1)
        aggregate[node] = self.linears[i - 1](aggregate[node])
        aggregate[node] = self.relu(aggregate[node])
      
      tmp = list(batchs[i])
      agg = aggregate[tmp[0]]
      for j in range(1, len(tmp)):
        agg = torch.cat((agg, aggregate[tmp[j]]), dim=0)

      agg = self.bns[i - 1](agg)

      for j, node in enumerate(tmp):
        fs[node] = agg[j].unsqueeze(0)

    z = fs[batchlist[0]]
    for i in range(1, len(batchlist)):
      z = torch.cat((z, fs[batchlist[i]]))
    
    result = self.softmax(z)

    return result

In [ ]:
lr = 0.1
epochs = 50
k = 3
n = 20
dims = [dataset.num_node_features]
diff = (dataset.num_node_features - dataset.num_classes) // k
for i in range(k):
  dims.append(dims[0] - diff * (i + 1))

dims.append(dataset.num_classes)

In [ ]:
model = Graphsage(neighbors, features, k, n, dims).to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.NLLLoss()

In [ ]:
model.train()
for epoch in range(epochs):
  acc = 0
  for x, y in train_loader:
    optimizer.zero_grad()
    out = model(x)
    loss = criterion(out, y)
    loss.backward()
    optimizer.step()
    _, pred = out.max(dim=1)
    acc += float(pred.eq(y).sum().item())
  print("epoch: {0}, loss: {1}, train acc: {2}".format(epoch, loss.item(), acc / data.train_mask.sum().item()))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


epoch: 0, loss: 2.304790735244751, train acc: 0.16428571428571428
epoch: 1, loss: 1.9278360605239868, train acc: 0.2857142857142857
epoch: 2, loss: 1.5891647338867188, train acc: 0.4857142857142857
epoch: 3, loss: 1.9449423551559448, train acc: 0.55
epoch: 4, loss: 1.570182204246521, train acc: 0.6
epoch: 5, loss: 1.131963849067688, train acc: 0.6357142857142857
epoch: 6, loss: 1.3880157470703125, train acc: 0.6571428571428571
epoch: 7, loss: 1.1769357919692993, train acc: 0.6785714285714286
epoch: 8, loss: 0.873522937297821, train acc: 0.7142857142857143
epoch: 9, loss: 0.8496752381324768, train acc: 0.7285714285714285
epoch: 10, loss: 0.7919709086418152, train acc: 0.7428571428571429
epoch: 11, loss: 1.0948494672775269, train acc: 0.7214285714285714
epoch: 12, loss: 0.5588100552558899, train acc: 0.75
epoch: 13, loss: 0.6127800345420837, train acc: 0.7857142857142857
epoch: 14, loss: 0.540549635887146, train acc: 0.7857142857142857
epoch: 15, loss: 0.6086921691894531, train acc: 0.78